# Projet Recherche Opérationnelle

Antoine GICQUEL \
Victor BERTRET \
4ème année Génie Mathématique \
Annéé universitaire 2020 - 2021

*Importation des packages nécessaires:*

In [2]:
from pulp import* #Package pulp : résolution de modèles PLNE
import numpy as np 
import random as random

## Introduction

Ce projet a été réalisé dans le cadre du module de Recherche Opérationnelle de notre formation d'ingénieur au département Génie Mathématique de l'INSA de Rennes. L'objectif de ce projet était d'étudier le fonctionnement algorithme de Branch-and-Bound appliqué au problème d'ordonnancement qui est le suivant. On considère un atelier de production devant usiner un ensemble $J = \{1,...,n\}$ de pièces. On suppose que l'atelier ne dispose que d'une machine (le problème peut tout à fait être étendu au cas où l'atelier dispose de plusieurs machines). Chaque pièce $j\in J$ nécessite $p_j$ unités de temps pour être usinée. De plus, à la date $t=0$, chaque pièce $j\in J$ doit être usinée avant la date $d_j$. Si ce n'est pas le cas et que la pièce $j$ est prête après la date $d_j$ alors il faudra payer une pénalité de $\omega_j$ par unité de temps de retard. On cherche à trouver l'ordre d'usinage des pièces minimisant la somme totale des pénalités de retard. \
Notons dés maintenant que ce problème peut être modélisé sous la forme d'un problème linéaire en nombres entiers que nous avons explicité ci-dessous :


**Variables:**

 * $\forall j \in J, r_j \ge 0$ est une variable réelle positive représentant le retard en unité de temps de la pièce $j$
 
 * $\forall j \in J, f_j \ge 0$ est une variable réelle positive représentant la date de fin d'usinage de la pièce $j$
 
 * $\forall (i,j)\in J^2$, $x_{i,j} \in \{0,1\}$ est une variable binaire prenant la valeur $1$ si la pièce $i$ passe avant la pièce $j$ et $0$ sinon

**Objectif:**

* On cherche à minimiser les pénalités de retard :

$$ \min{\sum_{j\in J}{\omega_j r_j}}$$

**Contraintes:**

* On impose la contrainte suivante pour calculer le retard de chaque pièce :
$$ r_j \ge f_j - d_j,\quad \forall j \in J$$

* On impose la contrainte suivatne pour calculer la date de fin d'usinage de chaque pièce :
$$ f_j = T_j + \sum_{i \in J \\ i\neq j}{T_i x_{ij}} \quad \forall j \in J$$

* On impose enfin une relation d'ordre total sur le passage des pièces :
$$ x_{ij} + x_{ji} = 1 \quad \forall (i,j) \in J^2, \: i < j$$
$$ x_{ik} \ge x_{ij} + x_{jk}-1\quad \forall (i,j,k) \in J^3, \: i<j, \: i<k, j\neq k$$

\
Nos implémentations ont été réalisé sous Python et nous avons fait appel (lorsque c'était nécessaire) au package PuLP pour résoudres des modèles de PLNE.

Nous avons dans un premier temps créé une structure de données permettant de représenter une instance du problème d'ordonnacement présenté ci-dessus. Nous avons ensuite créée une structure de données générique pour l'arbre d'énumération qui sera utilisé dans l'algorithme de Branch-and-Bound. Avec celle-ci, nous avons par la suite, implémenté l'algorithme de Branch-and-bound de manière générique en fragmentant ses différentes spécificités dans des sous-fonctions, ceci nous a notament permis de tester plusieurs méthodes pour chacune d'entre elles et ainsi comparer leurs performances sur des instances du problème d'ordonnancement. Les résultats et analyses de nos tests sont détaillés dans ce notebook. Enfin nous avons programmé des fonctions permettant de visualiser le parcours de l'arbre de branchement de l'algorithme de Branch-And-Bound selon les méthodes utilisées. Nous présenterons également une résolution de l'extension du problème d'ordonnancement dans laquelle on ajoute des contraintes de précédence entre certaines paires de pièces à usiner.

## I - Structure de données générique pour le problème d'ordonnancement

Afin de représenter les caractéristique du problème d'ordonnancement (celui présenté en introduction), nous avons choisi de défini une classe *Ordonnancement* en Python. Nous devrons donc créer un objet issu de la classe Ordonnacement à chaque fois que l'on souhaitera traiter une instance du problème d'ordonnancement.\
Le code de la classe est donné ci-dessous. Chaque problème d'ordonnancement contiendra les attributs suivants:
* **nb_piece** : le nombre de pièces contenues dans le problème (qui seront indexés de 0 à *nb_piece*-1)
* **unite_temps** : une liste contenant les unités de temps $T_j$ nécessaires pour l'usinage de chaque pièce $j$ (ces valeurs sont rangées dans l'ordre des indices des pièces) 
* **deadlines** : une liste contenant les dates limites (deadlines) d'usinage $d_j$ de chaque pièce $j$ (ces valeurs sont rangées dans l'ordre des indices des pièces)
* **penalites** : une liste contenant les valeurs des pénalités de retard $\omega_j$ par unité de temps de chaque pièce $j$ (ces valeurs sont rangées dans l'ordre des indices des pièces)
* **contraintesPrecedence** : un tableau associatif qui gardera en mémoire les contraintes de précédences imposées sur les pièces en indiquant pour chaque pièce présente en entrée, les autres pièces qui doivent être usinées avant celle-ci (Nous traiterons les contraintes de précédences sur les pièces dans un second temps).

De même, la classe *Ordonnancement* contient les fonctions suivantes:

* **ajoutePieces(...)** : fonction permettant d'ajouter un ensemble de pièces au problème. Elle prend en paramètre une liste contenant les unités de temps des pièces à ajouter, une liste contenant les deadlines des pièces à ajouter et une liste contenant les pénalités par unité de temps des pièces à ajouter (notons que les 3 listes passées en paramètre doivent avoir la même longueur).
* **ajouterContraintePrecedence(...)** : fonction permettant d'ajouter des contraintes de précédence sur l'ordre de passage des pièces. Elle prend en paramètre la pièce à ajouter et une liste contenant les pièces devant passer avant cette dernière
* **afficherProbleme()** : fonction permettant d'afficher les données du problème (valeurs de $T_j$, $\omega_j$ et $d_j$ $\forall j \in J$ ainsi que les éventuelles contraintes de précédence).
* **problemeAleatoire(...)** : fonction permettant de créer une instance aléatoire contienant n (donné en paramètre et fixé par défaut à 10) objets. Notons que cette fonction créé une nouvelle instance en écrasant la précédente contenue dans l'objet.
* **resolutionPLNE(...)** : fonction permettant de modéliser le problème courant sous la forme du PLNE présenté en introduction et qui le résout en affichant son résultat (et en indiquant éventuellement si l'instance proposée est non-réalisabe). Elle prend en paramètre un booléen permettant d'afficher ou non la valeur des variables après la résolution.

In [3]:
class Ordonnancement:
    
    def __init__(self):
        self.nb_piece=0 #Nombre de pièces
        self.unite_temps=[] #Unités de temps nécessaires pour les pièces
        self.deadlines=[] #Deadlines pour chaque pièce
        self.penalites=[] #Pénalités pour les retards de chaque pièce
        self.contraintesPrecedence = {} #Contraintes de précédence entre les pièces
        
    def ajouterPieces(self,liste_ut,liste_d,liste_p):
        #On vérifie qu'il ne manque pas d'informations
        if(len(liste_ut) == len(liste_d) == len(liste_p)):
            self.nb_piece += len(liste_ut) #Augmentation du nombre de pièces
            self.unite_temps += liste_ut #Ajout des unités de temps des nouvelles pièces
            self.deadlines += liste_d #Ajout des deadlies des nouvelles pièces
            self.penalites += liste_p #Ajout des penalites des nouvelles pièces
        else:
            print("Attention les listes doivent être de même longueur")
    
    #Fonction permettant d'imposer que les pices de la liste 'pieces_precedentes' passent avant 'piece'
    def ajouterContraintePrecedence(self,piece,pieces_precedentes):
        if(piece >= self.nb_piece):
            #La pièce doit être dans la liste pour lui imposer des contraintes de précédence
            print("Attention ! La pièce",piece,"n'a pas encore été ajoutée")
        else:
            #On ajoute les contraintes de précédence
            if piece not in self.contraintesPrecedence:
                self.contraintesPrecedence[piece] = pieces_precedentes
            else:
                for p in pieces_precedentes:
                    self.contraintesPrecedence[piece].append(p)
    
    #Fonction permettant d'afficher les données du problèmes créé
    def afficherProbleme(self):
        print("\nAFFICHAGE DU PROBLEME:\n")
        print("############### PIECES ##############")
        for i in range(self.nb_piece):
            print("Piece n°",i,": [Unités de temps nécessaires: ",self.unite_temps[i],", Deadline: ",self.deadlines[i],", Pénalité par unité de temps de retard: ",self.penalites[i],"]")
        print("###### CONTRAINTES PRECEDENCES ######")
        if(len(self.contraintesPrecedence) ==  0):
            print("Pas de contraintes de précédence")
        else:
            for i in self.contraintesPrecedence:
                for j in self.contraintesPrecedence[i]:
                    print("La pièce",j,"doit passer avant la pièce",i)
    
    #Fonction permettant de créer une instance de n pièces aux valeurs aléatoire 
    def problemeAleatoire(self,n = 10):
        self.nb_piece = n #Nombre de pièces
        self.unite_temps = [random.randint(1,20) for i in range(n)] #Unités de temps
        self.deadlines = [random.randint(20,40) for i in range(n)] #Deadlines
        self.penalites = [random.randint(1,10) for i in range(n)] #Pénalités
        
        
    def resolutionPLNE(self,afficheTout=0):
        #Création du modèle:
        modele = LpProblem("Ordonnancement",LpMinimize)
        #Ensembles des pièces
        J = list(range(self.nb_piece))
        #Variables:
        r = LpVariable.dicts("r",J,0,None,LpContinuous) #Retard de chaque piece
        f = LpVariable.dicts("f",J,0,None,LpContinuous) #Date de fin d'usinage de chaque piece
        x = LpVariable.matrix("x",(J,J),0,1,LpInteger) #Ordre de passage des pieces
        #Objectif:
        modele += pulp.lpSum([self.penalites[j]*r[j] for j in J]) #Minimisation des pénalités de retard
        #Contraintes:
        
        for j in J:
            modele += r[j] >= f[j] - self.deadlines[j] #Calcul du retard
        
        for j in J:
            modele += f[j] == self.unite_temps[j] + lpSum([self.unite_temps[i]*x[i][j] for i in J if i != j]) #Calcul des dates de fin d'usinage
        
        for j in J:
            modele += x[j][j] == 0 #Une pièce ne peut pas passer avant elle-même
        
        for j in J:
            for i in range(j):
                modele += x[i][j] + x[j][i] == 1 #Impose l'ordre total sur la relation d'ordre
        
        for j in J:
            for i in J:
                for k in J:
                    if(i<j and i<k and j != k):
                        modele += x[i][k] >= x[i][j] + x[j][k]- 1 #Impose la transitivité sur la relation d'ordre
        
        for j in self.contraintesPrecedence:
            for i in self.contraintesPrecedence[j]:
                modele += x[i][j] == 1 #Impose la contrainte de précédence : i doit passer avant j
                
        print("\nRESOLUTION DU PROBLEME:\n")
        #Résolution du problème:
        print("Solve with CBC")
        modele.solve(pulp.PULP_CBC_CMD())
        #Affichage de la solution
        print("Status:",LpStatus[modele.status])
        print("Optimal value =",value(modele.objective))
        #Si option selectionnée, on affiche toutes variables
        if(afficheTout):
            print(modele)
            for v in modele.variables():
                print(v.name,"=",v.varValue)
        #Si la solution est optimale, on affiche le résultat
        if(LpStatus[modele.status] == "Optimal"):
            print("\nSOLUTION OBTENUE PLNE:\n")
            print("Total des pénalités:",value(modele.objective))
            #Reconstruction de l'ordre des pièces
            Ordre = []
            k = self.nb_piece - 1
            while(k >= 0):
                for i in J:
                    if(sum([value(x[i][j]) for j in J]) == k):
                        Ordre.append(i)
                k -= 1
            print("Ordre d'usinage des pièces:",Ordre)
        else:
            print("Erreur lors de la résolution")

Pour illustrer le fonctionnement de cette classe, nous allons considérer l'exemple suivant composés de $n=4$ pièces (on considérera tout au long du projet $J=\{0,...,n-1\}$ pour les éléments de $J$ coïncident avec les indices des pièces dans les listes) ainsi que les valeurs correspondantes de $\omega_j$, $T_j$ et $d_j$ pour chaque pièce $j$.

![title](Exemple.PNG)

On commence par construire un objet correspondant au problème d'ordonnancement associé au tableau ci-dessus:

In [4]:
Exemple = Ordonnancement()
P = [4,5,3,5] #Pénalités
T = [12,8,15,9] #Unités de temps nécessaires
D = [16,26,25,27] #Deadlines
Exemple.ajouterPieces(T,D,P)
Exemple.afficherProbleme()


AFFICHAGE DU PROBLEME:

############### PIECES ##############
Piece n° 0 : [Unités de temps nécessaires:  12 , Deadline:  16 , Pénalité par unité de temps de retard:  4 ]
Piece n° 1 : [Unités de temps nécessaires:  8 , Deadline:  26 , Pénalité par unité de temps de retard:  5 ]
Piece n° 2 : [Unités de temps nécessaires:  15 , Deadline:  25 , Pénalité par unité de temps de retard:  3 ]
Piece n° 3 : [Unités de temps nécessaires:  9 , Deadline:  27 , Pénalité par unité de temps de retard:  5 ]
###### CONTRAINTES PRECEDENCES ######
Pas de contraintes de précédence


Une fois ce problème créé, nous pouvons maintenant le résoudre avec le modèle PLNE associé en appelant la fonction  *resolutionPLNE(...)*.

In [5]:
Exemple.resolutionPLNE() #Résolution 


RESOLUTION DU PROBLEME:

Solve with CBC
Status: Optimal
Optimal value = 67.0

SOLUTION OBTENUE PLNE:

Total des pénalités: 67.0
Ordre d'usinage des pièces: [0, 1, 3, 2]


On obtient que la valeur optimale du problème est  $z^*=67$ (valeur optimale de la fonction objectif du PLNE) et un ordre correspondant à cette solution est le suivante $x^* = (0,1,3,2)$ (Il peut y en avoir d'autre).\
Nous pourrons ainsi vérifier plus loin si notre algorithme de Branch-And-Bound nous fournit également cette valeur optimale.

## II - Structure de données générique pour l'arbre d'énumération de l'algorithme de Branch-And-Bound

Après avoir implémenté la structure de données génériques pour le problème d'ordonnancement, nous nous sommes intéressé à la mise en place de l'algorithme de Branch-and-Bound. Premièrement, nous devions créer une structure de données génériques pour l'arbre d'énumération.

Pour cela, nous avons séparé la création de l'arbre en 2 étapes. Tout d'abord, nous avons crée une classe Noeud. Cette classe noeud regroupe les informations dont nous avons besoin pour chaque noeud :

* **description** : titre du noeud.


* **position** : elle permet de savoir si le noeud est à l'extrémité de l'arbre ( feuille) ou non ( branche ).


* **indice_pere** : elle nous donne la position du noeud père dans la liste des noeuds que nous allons vous présenter par la suite dans une deuxième classe. Le noeud racine prendra la valeur -1.


* **info** : cette variable permet de décrire un noeud. Dans notre cas, elle contiendra, la plupart du temps, l'ordre des pièces déjà fixées.

Voici l'implémentation de la classe Noeud en python :

In [6]:
class Noeud:
    def __init__(self,description,pos,info =[],indice=-1):
        self.description=description
        self.position = pos #branche : 0 ou une feuille : 1
        self.indice_pere = indice #-1 si noeud racine
        self.info = info

Ensuite, dans un second temps, nous avons crée la classe ArbreEnumeration. Cette classe permet de définir l'arbre qui va être utilisé par la suite dans l'algorithme de Branch-and-Bound. Celle-ci va donc permettre de stocker tous les noeuds de l'arbre. 
Chaque instance de la classe est constituée des attributs suivants :

* **nombre_noeuds** : nombre de noeuds de l'arbre crée suite à l'algorithme B&B


* **nombre_noeuds_non_traites** : nombre de noeuds qui n'ont pas encore été traités


* **noeuds** : liste regroupant tous les noeuds


* **indice_noeuds_non_traites** : liste regroupant les indices de la liste noeud des noeuds qui n'ont pas encore été traités



* **primal_bound** : borne primale du problème


Voici l'implémentation de la classe ArbreEnumeration en python : 

In [7]:
class ArbreEnumeration:
    
    def __init__(self,p1):
            self.nombre_noeuds=1
            self.nombre_noeuds_non_traites = 1
            self.noeuds = [Noeud("Problème initiale",False)]
            self.indice_noeuds_non_traites= [0]
            self.primal_bound=p1

La structure de l'arbre d'énumération est maintenant implémentée. 

## III - Implémentation de l'algorithme de Branch-And-Bound dans le cadre du problème d'ordonnancement

Nous devions maintenant implémenter l'algorithme de Brand-and-Bound. Lorsque nous voulons appliquer l'algorithme B&B à un problème particulier, dans notre cas le problème d'ordonnancement, nous devons spécifier :


* **calcul de la borne duale** : Nous devons spécifier à l'algorithme comment calculer la borne duale de chaque noeud.


* **règle de branchement** : Nous devons spécifier la règle de branchement, c'est à dire spécifier à l'algorithme comment créer les noeuds enfants.


* **borne primale initiale** : Nous devons donner à l'algorithme une première borne primale initiale. Celle-ci n'est pas obligatoire mais elle permet d'accélérer grandement l'algorithme. 


* **méthode d'exploration** : Nous devons spécifier à l'algorithme comment parcourir l'arbre d'énumération.

Ainsi, notre objectif était de créer l'implémentation la plus générique possible et créer des variantes de l'algorithme en modifiant les élements que nous venons de présenter au dessus. Pour cela, notre fonction B&B prend en paramètre l'instance du problème, la borne primale initiale, le calcul de la borne duale, la règle de branchement ainsi que la méthode d'exploration.

Ensuite, afin d'implémenter l'algorithme de B&B, nous avons suivi le pseudocode présenté ci-dessous : 

![image.png](pseudocode-B&B.PNG)

Pour simplifier l'algorithme, nous avons crée quelques fonctions externes. Premièrement, nous avons crée une fonction **recuperationNoeud**. Celle-ci a pour objectif de récupérer un noeud de la liste des noeuds non traités et de calculer sa borne duale (ligne 5 à 7). 


Pour cela, la méthode prend en paramètres les éléments suivants : **arbre**, l'arbre d'énumération associé à l'algorithme ( instance de la classe ArbreEnumeration ), **explo**, la méthode d'exploration, **k**, le nombre d'itération, **borne_duale**,  méthode pour calculer la borne duale et **instance**, le probleme que nous voulons résoudre


En retour, la méthode peut ainsi renvoyée les éléments suivants :


* **Pk** : l'indice du noeud qui est en train d'être traité


* **zD** : borne duale du noeud indicé par Pk


* **isOptimal** : variable booléenne indiquant si le noeud est une feuille ( vrai ) ou bien une branche ( faux )


* **solution** : solution réalisable du noeud si le noeud est un noeud feuille

Voici son implémentation en python :

In [10]:
def recuperationNoeud(arbre,explo,k,borne_duale,instance): 
    Pk = arbre.indice_noeuds_non_traites.pop()
    
    arbre.nombre_noeuds_non_traites-=1
    
    #Calcul de la borne dual de Pk
    zD, isOptimal, solution  = borne_duale(arbre.noeuds[Pk],instance)
    #isOptimal == pos du noeud
    
    return Pk, zD, isOptimal, solution

Deuxièmement, nous avons créé une deuxième fonction externe : **exploration** (ligne 13). Pour l'instant, cette fonction appelle juste la fonction de la méthode d'exploration qui est donnée en paramètre. Elle a donc pour but d'organiser la liste des noeuds non traités selon la méthode d'exploration.

Celle-ci prend donc les mêmes paramètres que la méthode d'exploration et un paramètre **explo** pour spécifier la règle d'exploration utilisé. Les autres paramètres sont : **arbre**, l'arbre d'énumération, **list_nodes**, liste des noeuds renvoyés par la règle de branchement, **borne_duale**, méthode de calcul de la borne duale et **instance**, le problème que nous voulons résoudre.

Voici son implémenttation en python : 

In [11]:
def exploration(explo,arbre,list_nodes,borne_duale,instance):
    explo(arbre,list_nodes,borne_duale,instance)

Nous pouvons maintenant vous présenter notre méthode pour l'algorithme de Branch-and-Bound. Comme nous l'avons déjà précisé précédemment, cette méthode prend en paramètre : **instance**, l'instance du problème que nous voulons résoudre, **primale**, une borne primale initiale, **borne_duale**, la méthode pour calculer la borne duale, **Branchement**, la méthode de branchement et **explo**, la méthode d'exploration.


En retour, elle renvoie la valeur optimale de l'objectif **arbre.primal_bound** , la solution optimale **Current_best_solution** et le nombre d'itérations de l'algorithme **k**.

Voici son implémentation en python : 

In [12]:
#Algorithme du branch-and-bound:
def branch_and_bound(instance,primale,borne_duale,Branchement,explo):
    
    #Calcul de la borne primale initiale et itialisation de la meilleure solution courante:
    p, Current_best_solution = primale(instance)
    
    #Création de l'arrbre d'énumération:
    arbre=ArbreEnumeration(p) 
    
    #Variable comptant le nombre d'itérations
    k = 1
    
    #Boucle principale
    while(arbre.indice_noeuds_non_traites != []):
        #Récupération du premier noeud Pk dans Q et de la borne duale du problème associé
        Pk, zD, isOptimal, solution = recuperationNoeud(arbre,explo,k,borne_duale,instance)
        
        #Disjonction des cas:
        if(zD >= arbre.primal_bound):
            #Si zD est plus grand que la borne primale courante, on élague (uniquement pour un problème de minimisation)
            pass
        elif(isOptimal):
            #Si zD est optimale pour Pk, on met à jour la borne primale:
            if zD < arbre.primal_bound:
                #On met à jour la meilleure solution courante:
                Current_best_solution = solution
                arbre.primal_bound = zD
        else:
            #Sinon branchement et ajout des noeuds fils à l'arbre (en fonction de la méthode d'exploration):
            list_nodes=Branchement(arbre,instance,Pk)
            exploration(explo,arbre,list_nodes,borne_duale,instance)
        k+=1
    return arbre.primal_bound, Current_best_solution, k

Nous allons par la suite pouvoir vous présenter les différentes méthodes que nous avons adopté afin de résoudre les problèmes. 

### a) Méthode 1 *( Du début à la fin )* 

Nous avons commencé par une première méthode très simple : nous décidons de fixer l'ordre de passage des pièces à chaque niveau. Nous devions définir au moins une méthode d'exploration, une règle de branchement, une méthode de calcul pour la borne duale et une borne primale initiale.

Commencons par la règle de branchement. Considérons que le noeud racine est le problème initial P, c'est à dire qu'aucune pièce n'est pour le moment fixée.


Nous commencons alors par créer un noeud pour chaque pièce en la fixant en 1ère position. Itérativement, au niveau k de l'arbre, nous créeons un noeud pour chaque pièce non usinée et nous l'usinons en k-ème position. 

Si nous prenons une instance avec 2 produits, voici l'arbre avec tous les noeuds possibles: 

![image.png](GrapheDebut.PNG)

ici rajouter explication branchement 1

In [ ]:
def Branchement1(arbre,instance,indice_pere):
    #recuperation du noeud que nous traitons
    noeud=arbre.noeuds[indice_pere]
    
    #initialisation de la liste qui va contenir les noeuds que nous allons creer
    new_node=[]
    
    #recuperation des pieces non usinees
    pieces_non_usinees = [i for i in range(instance.nb_piece) if i not in noeud.info]
    
    #si il reste que deux pieces non usinees alors c'est un noeud fils
    if(len(pieces_non_usinees) == 2):
        position = True
    else:
        position = False
        
    #creation des nouveaux noeuds
    for i in pieces_non_usinees:
        new_info = noeud.info.copy()
        new_info.append(i)
        des = "P"
        for piece in noeud.info:
            des += str(piece)
        des += str(i)
        new_node.append(Noeud(des,position,new_info,indice_pere))
        
    #nous retournons la liste des nouveaux noeuds
    return new_node

### b) Méthode 2 *(En partant de la fin)*

![image.png](GrapheFin.PNG)

### c) Nouvelles méthodes de parcours

### d) Nouvelles méthodes pour les bornes primales

### e) Analyse des résultats

## IV - Visualisation du parcours de l'arbre de branchement lors de l'exécution de l'algorithme de Branch-And-Bound

    TO DO

## V - Ajout de contraintes de précédence sur les pièces au problème d'ordonnancement 

    TO DO

## Conclusion 

    TO DO